In [3]:
import openrouteservice as ors
import folium
from pandas import DataFrame
import pandas as pd
import math
import time

In [24]:
client = ors.Client(key='5b3ce3597851110001cf6248e87222039de944878943e770cf957d7d')

In [9]:
cur_dir = '../data/curated/'
raw_dir = '../data/raw/'

In [10]:

rent = pd.read_csv("../rent_data/renew(15)")
school = pd.read_csv(f"{cur_dir}school_df_21_preprocessed.csv")
mall =  pd.read_csv(f'{raw_dir}mall_info.csv')

In [11]:
def mapping(coordinates, client):
    map = folium.Map(location=[37.50, 144.58], tiles='cartodbpositron', zoom_start=13)
    route = client.directions(
        coordinates=coordinates,
        profile='driving-car',
        format='geojson',
        validate=False,
        )
    folium.PolyLine(locations=[list(reversed(coord)) for coord in route['features'][0]['geometry']['coordinates']]).add_to(map)
    return route, map

In [50]:
def cal_distance(coordinates, client):
    matrix = client.distance_matrix(
    locations=coordinates,
    profile='driving-car',
    metrics=['distance', 'duration'],
    validate=False,
    )
    time.sleep(2)
    return matrix['durations'], matrix['distances']
# the unit of distance is m, the unit of durations is seconds



In [13]:
def times(len):
    times_i = int(math.sqrt(3500))-1
    times = math.ceil(len/times_i)
    times_function = math.ceil((len+times)/times_i)
    return times_i, times_function

In [48]:
def get_coordinates(rent, school):
    rent_address = []
    school_name = []
    min_distance = []
    min_duration = []
    for i in range(len(rent)):
        coordinates = []
        distances = []
        durations = []
        for j in range(len(school)):
            coordinates.append([school["X"][j], school["Y"][j]])
        for k in range(times(len(school))[1]):
            coor_part = []
            coor_part.append([rent["longitude"][i], rent["latitude"][i]])
            coor_part.extend(coordinates[k*times(len(coordinates))[0]:(k+1)*times(len(coordinates))[0]])
            matrix = cal_distance(coor_part, client)
            distances = distances + matrix[1][0][1:]
            durations = durations + matrix[0][0][1:]
        index = distances.index(min(distances))
        rent_address.append(rent['listingSlug'][i])
        school_name.append(school['School_Name'][index])
        min_distance.append(distances[index])
        min_duration.append(durations[index])
    return rent_address, school_name, min_distance, min_duration

In [57]:
def get_coordinates(rent, school):
    rent_address = []
    school_name = []
    min_distance = []
    min_duration = []
    for i in range(len(rent)):
        coordinates = []
        distances = []
        durations = []
        coordinates.append([rent["longitude"][i], rent["latitude"][i]])
        for j in range(len(school)):
            coordinates.append([school["X"][j], school["Y"][j]])
            
        for k in range(times(len(school))[1]):
            coor_part = []
            coor_part.append([rent["longitude"][i], rent["latitude"][i]])
            coor_part.extend(coordinates[k*times(len(coordinates))[0]:(k+1)*times(len(coordinates))[0]])
            
        distances.append(cal_distance(coor_part, client)[1])
        durations.append(cal_distance(coor_part, client)[0])
        
        index = distances.index(min(distances))
        rent_address.append(rent['listingSlug'][i])
        school_name.append(school['School_Name'][index])
        min_distance.append(distances[index])
        min_duration.append(durations[index])
    
    return rent_address, school_name, min_distance, min_duration
    
        
        
            

In [58]:
def data_frame(rent, school):
    #get dataframe
    data = {'rent_address': get_coordinates(rent, school)[0], 'school_name': get_coordinates(rent, school)[1], \
        'min_distance': get_coordinates(rent, school)[2], 'min_duration': get_coordinates(rent, school)[3]}
    df = DataFrame(data)
    return df

df = data_frame(rent, school)
df.to_csv(f"{cur_dir}distances_durations_school21.csv", index=False)

ApiError: 400 ({'error': {'code': 6003, 'message': "Parameter 'locations' has incorrect value or format."}, 'info': {'engine': {'version': '6.7.0', 'build_date': '2022-02-18T19:37:41Z'}, 'timestamp': 1663308644784}})

In [60]:
m = folium.Map(location=[37.50, 144.58], tiles='cartodbpositron', zoom_start=13)

coordinates = []
for i in range(10):
    coordinates.append((school["X"][i], school["Y"][i]))

matrix = client.distance_matrix(
    locations=coordinates,
    profile='driving-car',
    metrics=['distance', 'duration'],
    validate=False,
)

for marker in coordinates:
    folium.Marker(location=list(reversed(marker))).add_to(m)


print("Distances in m: {}".format(matrix['locations']))

KeyError: 'locations'

In [30]:
a = []

for i in range(len(school)):
    a.append([school["Y"][i], school["X"][i]])
    
a

[[-38.617713, 146.666601],
 [-38.386281, 142.590393],
 [-37.084502, 143.475649],
 [-36.901368, 145.234722],
 [-37.742675, 145.21398],
 [-38.170672, 144.342099],
 [-37.67302961, 144.4431207],
 [-37.56401278, 143.853936],
 [-36.129214, 146.88982],
 [-38.044094, 145.354417],
 [-37.43852098, 143.3817175],
 [-37.968608, 145.054635],
 [-38.347938, 142.650844],
 [-37.838424, 144.878345],
 [-37.53087, 142.039893],
 [-38.337224, 143.598336],
 [-37.066595, 144.222329],
 [-37.092935, 144.204662],
 [-37.42411355, 143.8915581],
 [-36.733207, 144.249857],
 [-37.602316, 145.269827],
 [-38.366338, 142.466796],
 [-38.339644, 144.959653],
 [-37.786804, 145.129673],
 [-36.128198, 144.747463],
 [-37.72498353, 145.1478042],
 [-36.720583, 144.254259],
 [-37.10500363, 144.3329025],
 [-38.001759, 145.258244],
 [-37.780674, 144.933923],
 [-37.798383, 144.892669],
 [-37.4908591, 144.5884416],
 [-37.951358, 145.02342],
 [-36.768862, 146.982923],
 [-38.197643, 144.347756],
 [-37.820112, 145.018377],
 [-37.754705,

In [46]:
train = pd.read_excel(f'{raw_dir}train_station_19to20.xlsx')
train.head()

,Fin_year,Stop_ID,Stop_name,Stop_lat,Stop_long,Pax_annual
0,FY19-20,19980,Melton Railway Station (Melton South),-37.703362,144.572217,653450
1,FY19-20,19981,Rockbank Railway Station (Rockbank),-37.729258,144.650630,57650
2,FY19-20,19982,Deer Park Railway Station (Deer Park),-37.777760,144.772307,395900
3,FY19-20,20020,Ardeer Railway Station (Ardeer),-37.783063,144.802193,46100
4,FY19-20,20287,Albury Railway Station (Albury (NSW)),-36.084263,146.924518,26300


In [40]:
def get_coor(rent, train):
    rent_address = []
    stop_name = []
    min_distance = []
    min_duration = []
    for i in range(10):
        coordinates = []
        distances = []
        durations = []
        for j in range(10):
            coordinates.append([train["Stop_long"][j], train["Stop_lat"][j]])
        for k in range(times(len(train))[1]):
            coor_part = []
            coor_part.append([rent["longitude"][i], rent["latitude"][i]])
            coor_part.extend(coordinates[k*times(len(coordinates))[0]:(k+1)*times(len(coordinates))[0]])
            matrix = cal_distance(coor_part, client)
            time.sleep(4)
            distances = distances + matrix[1][0][1:]
            durations = durations + matrix[0][0][1:]
        index = distances.index(min(distances))
        rent_address.append(rent['listingSlug'][i])
        stop_name.append(train['Stop_name'][index])
        min_distance.append(distances[index])
        min_duration.append(durations[index])
    return rent_address, stop_name, min_distance, min_duration


"""
function: put the required columns into a dataframe
param datas: dataframe
"""
def data_frame(rent, train):
    #get dataframe
    data = {'rent_address': get_coor(rent, train)[0], 'stop_name': get_coor(rent, train)[1], 'min_distance': get_coor(rent, train)[2], 'min_duration': get_coor(rent, train)[3]}
    df = DataFrame(data)
    return df

df = data_frame(rent, train)

ApiError: 400 ({'error': {'code': 6003, 'message': "Parameter 'locations' has incorrect value or format."}, 'info': {'engine': {'version': '6.7.0', 'build_date': '2022-02-18T19:37:41Z'}, 'timestamp': 1663305368433}})